In [1]:
import tensorflow
from tensorflow import keras 
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Conv2D,MaxPooling2D, BatchNormalization, Activation,Dropout
from tensorflow.keras.layers import AveragePooling2D, Input, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
import numpy as np
import os

In [2]:
## Dataset Load
## x,y split / test, train split
cifar_dataset = tensorflow.keras.datasets.cifar10
(train_x,train_y),(test_x,test_y) = cifar_dataset.load_data()

In [3]:
## Preprocess
# One Hot encoded output
train_y = to_categorical(train_y,num_classes=10)
test_y = to_categorical(test_y,num_classes=10)

In [4]:
## Standardise
# Mean = 0
# Var = 1
mean = np.mean(train_x)
stdDev = np.std(train_x)
train_x = (train_x - mean) / stdDev
test_x = (test_x - mean) / stdDev

### ResNet Block
Option 1:  
Conv -> BatchNormalisation -> Activation  
Option 2:  
Conv -> Activation -> BatchNormalisation  

In [5]:
conv_first = True
def resNet_layer(input,num_filters=16,kernel_size=3,strides=1,activation='relu',batch_normalisation=True):
    conv=Conv2D(num_filters, 
                kernel_size=(kernel_size,kernel_size), 
                strides=(strides,strides), 
                padding='same', 
                kernel_initializer='he_normal')
    x = input
    if conv_first:
        x = conv(x)
        if(batch_normalisation):
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if(batch_normalisation):
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x;   

### ResNet 34 Layers MiniVersion

In [6]:
def resNet34(input_shape,num_classes=10):
    start_channel = 16
    block_count = [3,1,3,1]
    

    inputs = Input(shape=input_shape)
    x = Conv2D(start_channel,
                (3,3),
                strides=(1,1),
                activation='relu',
                padding='same',
                kernel_initializer='he_normal',
                bias_initializer='zeros'
                )(inputs)
    
    stride_x = 2
    for stack in block_count:
        if(stride_x == 2):
            stride_x = 1
        else:
            stride_x = 2
        for block in range(stack):
            y = resNet_layer(input = x, num_filters=start_channel, strides=stride_x)
            z = resNet_layer(input=y,num_filters=start_channel,strides=1,activation=None)
            x = resNet_layer(input=x,num_filters=start_channel,kernel_size=1,strides = stride_x,activation=None,batch_normalisation=None)
            x = keras.layers.add([x,z])
            x = Activation('relu')(x)
            x = Dropout(0.3)(x)
        start_channel *= 2

    x = AveragePooling2D(pool_size=(2,2))(x)
    y = Flatten()(x)
    y = Dropout(0.2)(y)
    y = Dense(64,
                activation='tanh',
                kernel_initializer='glorot_uniform',
                bias_initializer='zeros'
                )(y)
    output = Dense(num_classes,activation='softmax',kernel_initializer='he_normal')(y)

    model = Model(inputs = inputs, outputs = output)
    return model

In [7]:
## Compile
input_shape = train_x.shape[1:] 
model = resNet34(input_shape=input_shape)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy']
              )

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 16)   2320        conv2d[0][0]                     
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 16)   64          conv2d_1[0][0]                   
______________________________________________________________________________________________

In [9]:
EPOCHS = 500
BATCH_SIZE = int(train_y.shape[0]/100)
print(BATCH_SIZE)

500


In [10]:
history = model.fit(train_x,train_y,
                    validation_data=(test_x,test_y),
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE,
                    verbose=2,
                    shuffle=True
)

Train on 50000 samples, validate on 10000 samples
Epoch 1/500
50000/50000 - 20s - loss: 2.2288 - acc: 0.1812 - val_loss: 2.2674 - val_acc: 0.1750
Epoch 2/500
50000/50000 - 19s - loss: 1.9721 - acc: 0.2704 - val_loss: 3.0994 - val_acc: 0.1012
Epoch 3/500
50000/50000 - 23s - loss: 1.7836 - acc: 0.3385 - val_loss: 2.8057 - val_acc: 0.1281
Epoch 4/500
50000/50000 - 27s - loss: 1.6668 - acc: 0.3776 - val_loss: 2.4545 - val_acc: 0.1964
Epoch 5/500


KeyboardInterrupt: 